# Interpreting Machine Learning models using LIME and SHAP
This notebook briefly shows how to use LIME and SHAP to explain any kind of AI model.

We will first cover LIME and then we will move on to SHAP.

## LIME
Import LIME and Vader

In [ ]:
# Import vader model and LIME for text
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from lime.lime_text import LimeTextExplainer

# Import numpy for formatting
import numpy as np

The object containing the trained model must be compatible with LIME. In particular, the object needs to have a `predict_proba` method.

Hence, we'll add a new method to the Vader class.

In [ ]:
# Declare a function that can score multiple texts
def predict_proba(self, texts):

  # Initialize empty list
  ret = []

  # Iterate over texts
  for text in texts:

    # Get negative score
    neg = self.polarity_scores(text).get('neg')

    # Return two outputs: (neg) and (1 - neg)
    ret.append([neg, 1 - neg])

  # Return predictions
  return np.array(ret)

# Add the function as a method
SentimentIntensityAnalyzer.predict_proba = predict_proba

# Instantiate model
vader = SentimentIntensityAnalyzer()

Now that our `vader` object has a `predict_proba` method, we can use a LIME instance to explain the prediction made on a text.

We will explain the prediction made by Vader on the following text:
> I HATE Mondays! I missed the bus and forgot my lunch...

In [ ]:
# Declare text
my_angry_text = 'I HATE Mondays! I missed the bus and forgot my lunch...'

# Instantiate explainer
explainer = LimeTextExplainer(class_names=['negative', 'not-negative'])

# Create explanation for text
explanation = explainer.explain_instance(
    my_angry_text,
    vader.predict_proba,
    num_features=4
)

# Show explanation
explanation.show_in_notebook()

## SHAP

Just like LIME, SHAP is also used to explain AI models. However, SHAP requires objects to have a very specific object structure as well as an entire sample to work.

SHAP is compatible with the most popular ML libraries, so instead of modifying VADER to match the structure expected by SHAP, we will use `sklearn` to fit a simple linear model on the diabetes dataset.

In [ ]:
# Import SHAP library
import shap

# Import pandas for data processing
import pandas as pd

# Import data and linear model from sklearn
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression

Training and fine-tuning are not the main topics of this article. We will therefore train a very simple model and pretend it's the black-box model we're trying to explain.

In [ ]:
# Load features as pandas dataframe
X = pd.DataFrame(
    data=load_diabetes()['data'],
    columns=load_diabetes()['feature_names']
)

# Load target
y = data=load_diabetes()['target']

# Instantiate model
model = LinearRegression()

# Fit model to data
model.fit(X=X, y=y)

As stated in the article, SHAP needs a sample to calculate each feature's shapley value. We will draw a sample of 200 observations and then print out the observed values of the 42nd patient as well as their predicted score.

Note that we are choosing the 42nd individual for no reason in particular! We could choose any other observation in the sample.

In [ ]:
# Draw sample of 200 observations
sample = shap.utils.sample(
    X=X,
    nsamples=200,
    random_state=0
)

# Pick 42nd observation from sample to explain
idx = 42

# Print 42nd observation
print(
    f'Observed features of observation number {idx}:\n',
    sample.iloc[[idx], :],
    sep=''
)

# Print predicted value made on 42nd observation
print(
    f'\n\nPredicted severity of the disease for patient number {idx}:\n',
    model.predict(sample.iloc[[idx], :]),
    sep=''
)

We will now instantiate the SHAP explainer using the sample we drew and plot a waterfall figure to understand how each of the 10 features affects the prediction made by the model on this particular observation.

In [ ]:
# Instantiate explainer
explainer = shap.Explainer(
    model=model.predict,
    masker=sample
)

# Calculate SHAP values using sample
shap_values = explainer(sample)

# Waterfall plot
shap.plots.waterfall(shap_values=shap_values[idx])

**NOTE:** Remember that explanations made by LIME and SHAP are **local**.

The plot tells us that `s5` is the most important feature and had a negative impact on the prediction; `s1` was the second most important feature and also had a negative effect on the prediction; etc.

In particular, `bmi` was the fifth most important feature and had a positive impact on the prediction made by the model! This means that for the 42nd patient, their high body mass index increased the predicted severity of their diabetes.

Each feature's SHAP value is represented by the arrow. To see where these numbers come from, we can print a *partial dependence plot*. In this example, we will focus on the `bmi` variable (though we could choose any other feature used by the model).

In [ ]:
# Make a partial dependence plot
shap.partial_dependence_plot(
    ind='bmi',
    model=model.predict,
    data=sample,
    model_expected_value=True,
    feature_expected_value=True,
    ice=False,
    shap_values=shap_values[idx:idx+1,:]
)

The above plot tells us that the SHAP value for `bmi` comes from the difference between $\mathbb{E}\big[f(X | bmi=0.015)\big]$ and $\mathbb{E}\big[f(X)\big]$.

The relationship between partial dependence plots and SHAP values only holds for linear additive models. However, the idea remains the same for other models.

In a nutshell, the SHAP value of a feature is an measure of what would happen to the prediction if the feature in question was not observable.